<a href="https://colab.research.google.com/github/Prashitraj/Algorithmic-Trading/blob/main/Quantitative_Value_Investing_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np
import pandas as pd
import requests
from scipy import stats
import math

In [29]:
stocks = pd.read_csv('/content/drive/MyDrive/StockAnalysis/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [30]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [31]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'RLT G/GDSANAQLMATOESEKANBE)L ( SC', 'calculationPrice': 'close', 'open': 125.38, 'openTime': 1645872702601, 'openSource': 'ficlaiof', 'close': 126.21, 'closeTime': 1644231699058, 'closeSource': 'iaofcfil', 'high': 125.37, 'highTime': 1636353418103, 'highSource': 'cm 15ayein l ritededepu', 'low': 123.23, 'lowTime': 1690960156411, 'lowSource': 't md pece5rii1 eduenlya', 'latestPrice': 121.82, 'latestSource': 'Close', 'latestTime': 'March 26, 2021', 'latestUpdate': 1695538714743, 'latestVolume': 98153193, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 126.63, 'delayedPriceTime': 1694729011639, 'oddLotDelayedPrice': 127.06, 'oddLotDelayedPriceTime': 1693271319962, 'extendedPrice': 124.1, 'extendedChange': -0.11, 'extendedChangePercent': -0.00091, 'extendedPriceTime': 1652323871884, 'previousClose': 122.32, 'previousVolume': 103447209, 'change': 0.64, 'changePercent': 0.00516, 'vol

In [32]:
price  = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

34.43

In [33]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [34]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
my_columns = ['Ticker', 'Price', 'PE Ratio', 'Number Of Shares to Buy']
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series([symbol,data[symbol]['quote']['latestPrice'],data[symbol]['quote']['peRatio'],'N/A'],index=my_columns),ignore_index=True)
final_dataframe

,Ticker,Price,PE Ratio,Number Of Shares to Buy
0,A,128.95,50.66,N/A
1,AAL,23.79,-1.25,N/A
2,AAP,193.43,26.81,N/A
3,AAPL,124.86,33.76,N/A
4,ABBV,110.23,40.72,N/A
...,...,...,...,...
500,YUM,109.81,37.7,N/A
501,ZBH,164.95,-251.83,N/A
502,ZBRA,494.67,51.6,N/A
503,ZION,57.91,8.8,N/A


In [35]:
final_dataframe.sort_values('PE Ratio', inplace = True )
final_dataframe = final_dataframe[final_dataframe['PE Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,PE Ratio,Number Of Shares to Buy
0,63,BIO,582.86,4.64,N/A
1,26,ALL,116.62,6.7,N/A
2,459,UNM,29.00,7.18,N/A
3,152,EBAY,62.40,7.72,N/A
4,17,AFL,52.97,8.03,N/A
5,108,COO,400.60,8.71,N/A
6,503,ZION,57.91,8.8,N/A
7,265,KIM,20.03,8.91,N/A
8,87,CE,154.77,9.4,N/A
9,271,KR,38.29,9.86,N/A


In [36]:
portfolio_size = input('Enter your portfolio size:')

try:
    val = float(portfolio_size)
except ValueError:
    print('Please enter an integer')

Enter your portfolio size:1000000


In [37]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number Of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe

,index,Ticker,Price,PE Ratio,Number Of Shares to Buy
0,63,BIO,582.86,4.64,34
1,26,ALL,116.62,6.7,171
2,459,UNM,29.00,7.18,689
3,152,EBAY,62.40,7.72,320
4,17,AFL,52.97,8.03,377
5,108,COO,400.60,8.71,49
6,503,ZION,57.91,8.8,345
7,265,KIM,20.03,8.91,998
8,87,CE,154.77,9.4,129
9,271,KR,38.29,9.86,522


In [38]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
pe_ratio = data[symbol]['quote']['peRatio']
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
ps_ratio = data[symbol]['advanced-stats']['priceToSales']
ev_to_ebitda = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['EBITDA']
ev_to_gp = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['grossProfit']
print(pe_ratio,pb_ratio,ps_ratio,ev_to_ebitda,ev_to_gp)

34.46 31.12 7 23.38930657892314 17.40826144557437


In [56]:
rv_columns = ['Ticker', 'Price', 'Number of Shares to Buy',
              'PE Ratio','PE Percentile',
              'PB Ratio','PB Percentile',
              'PS Ratio','PS Percentile',
              'EV_TO_EBITDA Ratio','EV_TO_EBITDA Percentile',
              'EV_TO_GP Ratio','EV_TO_GP Percentile',
              'RV SCORE']
rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        pe_ratio = data[symbol]['quote']['peRatio']
        pb_ratio = data[symbol]['advanced-stats']['priceToBook']
        ps_ratio = data[symbol]['advanced-stats']['priceToSales']
        try:
            ev_to_ebitda = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['EBITDA']
        except:
            ev_to_ebitda = np.NaN
        try:
            ev_to_gp = data[symbol]['advanced-stats']['enterpriseValue']/data[symbol]['advanced-stats']['grossProfit']
        except:
            ev_to_gp = np.NaN
        rv_dataframe = rv_dataframe.append(pd.Series(
            [symbol,data[symbol]['quote']['latestPrice'],'N/A',
             pe_ratio,'N/A',
             pb_ratio,'N/A',
             ps_ratio,'N/A',
             ev_to_ebitda,'N/A',
             ev_to_gp,'N/A',
             'N/A'],index=rv_columns),ignore_index=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV_TO_EBITDA Ratio,EV_TO_EBITDA Percentile,EV_TO_GP Ratio,EV_TO_GP Percentile,RV SCORE
0,A,126.66,N/A,49.35,N/A,8.01,N/A,7.24,N/A,30.665509,N/A,13.435138,N/A,N/A
1,AAL,23.81,N/A,-1.29,N/A,-2.17,N/A,0.871,N/A,-4.367339,N/A,2.318925,N/A,N/A
2,AAP,195.41,N/A,26.9,N/A,3.47,N/A,1.24,N/A,12.785389,N/A,2.815047,N/A,N/A
3,AAPL,126.42,N/A,34.58,N/A,31.96,N/A,7.11,N/A,25.335770,N/A,18.793993,N/A,N/A
4,ABBV,108.30,N/A,39.5,N/A,15,N/A,4.28,N/A,13.665192,N/A,8.370841,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.80,N/A,38.22,N/A,-4.24,N/A,5.77,N/A,24.982022,N/A,10.400046,N/A,N/A
501,ZBH,164.67,N/A,-240.85,N/A,2.77,N/A,4.86,N/A,31.060916,N/A,8.597894,N/A,N/A
502,ZBRA,478.95,N/A,53.41,N/A,12.46,N/A,5.84,N/A,35.398835,N/A,13.573791,N/A,N/A
503,ZION,58.11,N/A,9.1,N/A,1.26,N/A,2.74,N/A,6.975190,N/A,2.937215,N/A,N/A


In [57]:
rv_dataframe[rv_dataframe.isnull().any(axis =1)]

,Ticker,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV_TO_EBITDA Ratio,EV_TO_EBITDA Percentile,EV_TO_GP Ratio,EV_TO_GP Percentile,RV SCORE
40,AON,236.480,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
42,APA,18.880,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,258.990,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.92,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,36.660,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.900,N/A,14.73,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,61.650,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,37.570,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,177.970,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2085.510,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [58]:
rv_dataframe = rv_dataframe.fillna(value = np.nan)
for column in ['PE Ratio','PB Ratio','PS Ratio','EV_TO_EBITDA Ratio','EV_TO_GP Ratio']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(),inplace = True)

In [59]:
from scipy.stats import percentileofscore as score
stats = ['PE', 'PB','PS','EV_TO_EBITDA','EV_TO_GP']

for row in rv_dataframe.index:
    for stat in stats:
        change_col = f'{stat} Ratio'
        percentile_col = f'{stat} Percentile'
        rv_dataframe.loc[row,percentile_col] = score(rv_dataframe[change_col], rv_dataframe.loc[row,change_col])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV_TO_EBITDA Ratio,EV_TO_EBITDA Percentile,EV_TO_GP Ratio,EV_TO_GP Percentile,RV SCORE
0,A,126.66,N/A,49.35,83.5644,8.01,73.0693,7.240,79.604,30.665509,86.3366,13.435138,77.4257,N/A
1,AAL,23.81,N/A,-1.29,15.2475,-2.17,4.55446,0.871,7.12871,-4.367339,3.9604,2.318925,8.51485,N/A
2,AAP,195.41,N/A,26.90,48.6139,3.47,44.1584,1.240,14.0594,12.785389,30.099,2.815047,11.4851,N/A
3,AAPL,126.42,N/A,34.58,66.5347,31.96,95.0495,7.110,78.8119,25.335770,77.6238,18.793993,91.2871,N/A
4,ABBV,108.30,N/A,39.50,75.5446,15.00,87.9208,4.280,58.1188,13.665192,35.0495,8.370841,48.9109,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.80,N/A,38.22,73.8614,-4.24,4.35644,5.770,72.0792,24.982022,76.8317,10.400046,66.1386,N/A
501,ZBH,164.67,N/A,-240.85,0.990099,2.77,35.2475,4.860,62.1782,31.060916,86.7327,8.597894,50.495,N/A
502,ZBRA,478.95,N/A,53.41,85.1485,12.46,84.7525,5.840,72.7723,35.398835,89.3069,13.573791,78.6139,N/A
503,ZION,58.11,N/A,9.10,17.4257,1.26,10.6931,2.740,38.9109,6.975190,6.33663,2.937215,12.0792,N/A


In [60]:
from statistics import mean 
for row in rv_dataframe.index:
    stat_percentiles = []
    for stat in stats:
        stat_percentiles.append(rv_dataframe.loc[row, f'{stat} Percentile'])
        rv_dataframe.loc[row,'RV SCORE'] = mean(stat_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV_TO_EBITDA Ratio,EV_TO_EBITDA Percentile,EV_TO_GP Ratio,EV_TO_GP Percentile,RV SCORE
0,A,126.66,N/A,49.35,83.5644,8.01,73.0693,7.240,79.604,30.665509,86.3366,13.435138,77.4257,80
1,AAL,23.81,N/A,-1.29,15.2475,-2.17,4.55446,0.871,7.12871,-4.367339,3.9604,2.318925,8.51485,7.88119
2,AAP,195.41,N/A,26.90,48.6139,3.47,44.1584,1.240,14.0594,12.785389,30.099,2.815047,11.4851,29.6832
3,AAPL,126.42,N/A,34.58,66.5347,31.96,95.0495,7.110,78.8119,25.335770,77.6238,18.793993,91.2871,81.8614
4,ABBV,108.30,N/A,39.50,75.5446,15.00,87.9208,4.280,58.1188,13.665192,35.0495,8.370841,48.9109,61.1089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.80,N/A,38.22,73.8614,-4.24,4.35644,5.770,72.0792,24.982022,76.8317,10.400046,66.1386,58.6535
501,ZBH,164.67,N/A,-240.85,0.990099,2.77,35.2475,4.860,62.1782,31.060916,86.7327,8.597894,50.495,47.1287
502,ZBRA,478.95,N/A,53.41,85.1485,12.46,84.7525,5.840,72.7723,35.398835,89.3069,13.573791,78.6139,82.1188
503,ZION,58.11,N/A,9.10,17.4257,1.26,10.6931,2.740,38.9109,6.975190,6.33663,2.937215,12.0792,17.0891


In [61]:
rv_dataframe.sort_values('RV SCORE',ascending=False,inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(inplace=True)
rv_dataframe

,index,Ticker,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV_TO_EBITDA Ratio,EV_TO_EBITDA Percentile,EV_TO_GP Ratio,EV_TO_GP Percentile,RV SCORE
0,339,NOW,511.00,N/A,871.39,99.604,35.12,95.6436,21.89,99.2079,176.798614,99.802,26.719447,98.2178,98.495
1,358,PAYC,379.85,N/A,152.33,97.4257,35.55,95.8416,27.51,99.802,95.741185,99.2079,30.735832,99.2079,98.297
2,236,IDXX,481.48,N/A,71.72,91.6832,67.40,97.4257,15.89,96.4356,53.547437,95.0495,26.338760,97.8218,95.6832
3,345,NVDA,528.95,N/A,78.15,92.8713,19.42,91.4851,19.40,98.6139,55.995688,95.8416,30.964997,99.4059,95.6436
4,150,DXCM,369.27,N/A,70.77,91.0891,18.88,91.0891,18.21,97.8218,90.143909,98.6139,25.932357,97.4257,95.2079
5,308,MKTX,526.10,N/A,68.02,90.297,21.20,93.6634,28.69,100,46.602356,92.0792,27.847128,98.6139,94.9307
6,239,ILMN,412.65,N/A,92.43,94.2574,12.88,85.3465,18.70,98.2178,93.830939,99.0099,25.406260,97.0297,94.7723
7,294,MA,371.42,N/A,59.49,87.3267,57.31,97.0297,23.87,99.604,41.195633,91.0891,22.733188,96.0396,94.2178
8,250,ISRG,764.96,N/A,86.66,93.8614,9.29,79.802,19.91,98.8119,61.651308,96.6337,28.443974,98.8119,93.5842
9,451,TYL,436.42,N/A,92.74,94.4554,8.74,78.4158,15.52,96.0396,86.717608,98.4158,30.454860,99.0099,93.2673


In [62]:
portfolio_size = input('Enter your portfolio size:')

try:
    val = float(portfolio_size)
except ValueError:
    print('Please enter an integer')

Enter your portfolio size:1000000


In [63]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for i in range(0,len(final_dataframe)):
    rv_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[i,'Price'])
rv_dataframe

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,Ticker,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV_TO_EBITDA Ratio,EV_TO_EBITDA Percentile,EV_TO_GP Ratio,EV_TO_GP Percentile,RV SCORE
0,339,NOW,511.00,39,871.39,99.604,35.12,95.6436,21.89,99.2079,176.798614,99.802,26.719447,98.2178,98.495
1,358,PAYC,379.85,52,152.33,97.4257,35.55,95.8416,27.51,99.802,95.741185,99.2079,30.735832,99.2079,98.297
2,236,IDXX,481.48,41,71.72,91.6832,67.40,97.4257,15.89,96.4356,53.547437,95.0495,26.338760,97.8218,95.6832
3,345,NVDA,528.95,37,78.15,92.8713,19.42,91.4851,19.40,98.6139,55.995688,95.8416,30.964997,99.4059,95.6436
4,150,DXCM,369.27,54,70.77,91.0891,18.88,91.0891,18.21,97.8218,90.143909,98.6139,25.932357,97.4257,95.2079
5,308,MKTX,526.10,38,68.02,90.297,21.20,93.6634,28.69,100,46.602356,92.0792,27.847128,98.6139,94.9307
6,239,ILMN,412.65,48,92.43,94.2574,12.88,85.3465,18.70,98.2178,93.830939,99.0099,25.406260,97.0297,94.7723
7,294,MA,371.42,53,59.49,87.3267,57.31,97.0297,23.87,99.604,41.195633,91.0891,22.733188,96.0396,94.2178
8,250,ISRG,764.96,26,86.66,93.8614,9.29,79.802,19.91,98.8119,61.651308,96.6337,28.443974,98.8119,93.5842
9,451,TYL,436.42,45,92.74,94.4554,8.74,78.4158,15.52,96.0396,86.717608,98.4158,30.454860,99.0099,93.2673
